# Practica 4

- Nombre: Abraham de Jesus Cisneros Jarquin
- correo: abraham.cisneros6728@alumnos.udg.mx

Requisitos mínimos:

● Funciones que generen trayectorias tipo Brownian Motion (BM), Correlated Random Walk (CRW) y Lévy Flight (LF).

● Cada una de las funciones deberá tomar como parámetros el numero de pasos, la
velocidad y posición inicial.

○ Además la funciones para CRW y LF deberán tomar como parámetro el coeficiente para la distribución Cauchy.

○ Por último, la función para LF también deberá también aceptar como parámetro
el exponente Lévy (alpha).

● Funciones para calcular las métricas Path length and Mean Squared Displacement.

# Modules

In [1]:
import numpy as np
import math
import panel as pn
import pandas as pd
import plotly.graph_objects as go
import panel.widgets as pnw
pn.extension('plotly')
import param
from scipy.stats import levy_stable
from scipy.stats import wrapcauchy
import holoviews as hv
pn.extension(sizing_mode= 'stretch_width')


# Classes

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

# Functions

In [3]:
# Trayectoria del Movimiento Browniano
def trayectoria_browniana_2d(pasos=1000, velocidad=6, s_pos=[0,0]):
    """
    Argumentos:
        pasos: Número de pasos.
        velocidad: Velocidad.
        posicion_inicial: Posición inicial.
    Retorna:
        df_trayectoria: Dataframe con la trayectoria del Movimiento Browniano.
    """
    # Inicializar el vector de velocidad
    velocidad = Vec2d(velocidad, 0)

    df_trayectoria = pd.DataFrame(columns=['x_pos', 'y_pos'])
    temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
    df_trayectoria = pd.concat([df_trayectoria, temp_df], ignore_index=True)

    for i in range(pasos - 1):
        angulo_giro = np.random.uniform(low=-np.pi, high=np.pi)
        velocidad = velocidad.rotated(angulo_giro)

        temp_df = pd.DataFrame([{'x_pos': df_trayectoria.x_pos[i] + velocidad.x,
                                 'y_pos': df_trayectoria.y_pos[i] + velocidad.y}])
        df_trayectoria = pd.concat([df_trayectoria, temp_df], ignore_index=True)
    # Retorna el dataframe
    return df_trayectoria


# Caminata Aleatoria Correlacionada
def crwv1(pasos=100, velocidad=6, s_pos=[0,0], exp_crw=0.7):
    """
    Argumentos:
        pasos: Número de pasos.
        velocidad: Velocidad.
        posicion_inicial: Posición inicial.
        exp_crw: Exponente de la Caminata Aleatoria Correlacionada.
    Retorna:
        df_crw: Trayectoria de la Caminata Aleatoria Correlacionada.
    """

    velocidad = Vec2d(velocidad, 0)
    df_crw = pd.DataFrame(columns=["x_pos", "y_pos"])
    temp_df = pd.DataFrame([{"x_pos": 0, "y_pos": 0}])
    df_crw = pd.concat([df_crw, temp_df], ignore_index=True)

    bm_2d = wrapcauchy.rvs(c=exp_crw, loc=0, size=pasos-1)

    for i in range(1, pasos):
        angulo_giro = bm_2d[i-1]
        angulo_giro = wrapcauchy.rvs(c=exp_crw, loc=0)
        velocidad = velocidad.rotated(angulo_giro)

        temp_df = pd.DataFrame([{"x_pos": df_crw.x_pos[i-1] + velocidad.x,
                                 "y_pos": df_crw.y_pos[i-1] + velocidad.y}])
        df_crw = pd.concat([df_crw, temp_df], ignore_index=True)

    return df_crw


# Vuelo de Levy
def lvfv1(pasos=100, velocidad=6, s_pos=[0,0], cauchy_lv=0.4, exp_lv = 1):
    """
    Argumentos:
        pasos: Número de pasos.
        velocidad: Velocidad.
        posicion_inicial: Posición inicial.
        cauchy_lv: Coeficiente Cauchy para el Vuelo de Levy.
        exp_lv: Exponente del Vuelo de Levy.
    Retorna:
        df_vuelo_levy: Dataframe con la trayectoria del Vuelo de Levy.
    """
    velocidad = Vec2d(velocidad, 0)
    std_mstps = 3
    df_vuelo_levy = pd.DataFrame(columns=["x_pos", "y_pos"])
    temp_df = pd.DataFrame([{"x_pos": 0, "y_pos": 0}])
    df_vuelo_levy = pd.concat([df_vuelo_levy, temp_df], ignore_index=True)
    i = 1
    while i < pasos:
        angulo_giro = wrapcauchy.rvs(c=cauchy_lv, loc=0)
        velocidad = velocidad.rotated(angulo_giro)
        n_pasos = abs(math.floor(levy_stable.rvs(alpha=exp_lv, beta=1, loc=std_mstps)))
        if (i + n_pasos > pasos):
            n_pasos = pasos - 1

        c_paso = 0
        while c_paso < n_pasos:
            temp_df = pd.DataFrame([{"x_pos": df_vuelo_levy.x_pos[i-1] + velocidad.x,
                                     "y_pos": df_vuelo_levy.y_pos[i-1] + velocidad.y}])
            df_vuelo_levy = pd.concat([df_vuelo_levy, temp_df], ignore_index=True)
            c_paso += 1
            i += 1

    return df_vuelo_levy


# Values

In [4]:
n_steps = pnw.IntInput(name='Number steps', value=50, step=1, start=0, end=1000)
speed = pnw.IntInput(name='Speed', value=6, step=1, start=0, end=10)
x_pos = pnw.FloatInput(name='Starting pos_x', value=0, step=1, start=0, end=10000)
y_pos = pnw.FloatInput(name='Starting pos_y', value=0, step=1, start=0, end=10000)
crw_exp = pnw.FloatInput(name='Cauchy Coefficient', value=0.6, step=0.1, start=0.1, end=1)
lv_exp = pnw.FloatInput(name='Levy exponent', value=0.4, step=0.1, start=0.1, end=1)


# Trajectories

In [5]:
# Brownian Motion
@pn.depends(n_steps, speed, x_pos, y_pos)
def plot_traj(n_steps, speed, x_pos, y_pos):
    fig_traj_bm = go.Figure()
    bm_df = trayectoria_browniana_2d(n_steps, speed, [x_pos, y_pos])

    fig_traj_bm.add_trace(
        go.Scatter3d(x=bm_df.x_pos,
                     y=bm_df.y_pos,
                     z=bm_df.index,
                     marker=dict(size=2),
                     line=dict(color='#EB89B5', width=2),
                     modes='lines',
                     name=f'BM={n_steps}',
                     showlegend=True))

    fig_traj_bm.update_traces(groupnorm='fraction',
                selector=dict(type='scatter'))
    return fig_traj_bm


# Cauchy coefficient
@pn.depends(n_steps, speed, x_pos, y_pos, crw_exp)
def plot_traj_crw(n_steps, speed, x_pos, y_pos, crw_exp):
    fig_traj_crw = go.Figure()
    crw_df = crwv1(n_steps=n_steps, speed=speed, s_pos=[x_pos, y_pos], crw_exp=crw_exp)

    fig_traj_crw.add_trace(
        go.Scatter3d(x=crw_df.x_pos,
                     y=crw_df.y_pos,
                     z=crw_df.index,
                     marker=dict(size=2),
                     line=dict(color='#EB89B5', width=2),
                     modes='lines',
                     name=f'CRW={crw_exp}',
                     showlegend=True))

    fig_traj_crw.update_traces(groupnorm='fraction',
                            selector=dict(type='scatter'))
    return fig_traj_crw


# levy exponent
@pn.depends(n_steps, speed, x_pos, y_pos, crw_exp, lv_exp)
def plot_traj_lv(n_steps, speed, x_pos, y_pos, crw_exp, lv_exp):
    fig_traj_lv = go.Figure()
    lv_df = lvfv1(n_steps, speed, s_pos=[x_pos, y_pos], lvf_cauchy=crw_exp, lvf_exp=lv_exp)

    fig_traj_lv.add_trace(
        go.Scatter3d(x=lv_df.x_pos,
                     y=lv_df.y_pos,
                     z=lv_df.index,
                     marker=dict(size=2),
                     line=dict(color='#EB89B5', width=2),
                     modes='lines',
                     name=f'LVF={lv_exp}',
                     showlegend=True))

    fig_traj_lv.update_traces(groupnorm='fraction',
                            selector=dict(type='scatter'))
    return fig_traj_lv


# Longitud de trayectoria Brownian Motion
@pn.depends(n_steps, speed, x_pos, y_pos)
def met_bm(n_steps, speed, x_pos, y_pos):
    fig_met_bm = go.Figure()
    bm_df = trayectoria_browniana_2d(n_steps, speed, [x_pos, y_pos])
    met_df = ([np.linalg.norm(bm_df.iloc[i-1]-bm_df.iloc[i]) for i in range(1, bm_df.shape[0])])
    pl_bm = np.cumsum(met_df)
    fig_met_bm.add_trace(go.Scatter(
        x = np.arange(len(pl_bm)),
        y = pl_bm,
        marker = dict(size=2),
        line = dict(color='#EB89B5', width=2),
        mode = 'lines',
        name = 'Path Lenght',
        showlegend = True))
    fig_met_bm.update_layout(title_text='Path Lenght')

    return fig_met_bm


# Longitud de trayectoria Cauchy coefficient
@pn.depends(n_steps, speed, x_pos, y_pos, crw_exp)
def met_crw(n_steps, speed, x_pos, y_pos, crw_exp):
    fig_met_crw = go.Figure()
    crw_df = crwv1(n_steps=n_steps, speed=speed, s_pos=[x_pos, y_pos], crw_exp=crw_exp)
    met_df = ([np.linalg.norm(crw_df.iloc[i-1]-crw_df.iloc[i]) for i in range(1, crw_df.shape[0])])
    pl_crw = np.cumsum(met_df)
    fig_met_crw.add_trace(go.Scatter(
        x = np.arange(len(pl_crw)),
        y = pl_crw,
        marker = dict(size=2),
        line = dict(color='purple', width=2),
        mode = 'lines',
        name = 'Path Lenght',
        showlegend = True))
    fig_met_crw.update_layout(title_text='Path Lenght')

    return fig_met_crw


# Longitud de trayectoria levy exponent
@pn.depends(n_steps, speed, x_pos, y_pos, crw_exp, lv_exp)
def met_lv(n_steps, speed, x_pos, y_pos, crw_exp, lv_exp):
    fig_met_lv = go.Figure()
    lv_df = lvfv1(n_steps, speed, s_pos=[x_pos, y_pos], lvf_cauchy=crw_exp, lvf_exp=lv_exp)
    met_df = ([np.linalg.norm(lv_df.iloc[i-1]-lv_df.iloc[i]) for i in range(1, lv_df.shape[0])])
    pl_lv = np.cumsum(met_df)
    fig_met_lv.add_trace(go.Scatter(
        x = np.arange(len(pl_lv)),
        y = pl_lv,
        marker = dict(size=2),
        line = dict(color='turquoise', width=2),
        mode = 'lines',
        name = 'Path Lenght',
        showlegend = True))
    fig_met_lv.update_layout(title_text='Path Lenght')

    return fig_met_lv


# MSD Brownian motion
@pn.depends(n_steps, speed, x_pos, y_pos)
def plot_bm_msd(n_steps, speed, x_pos, y_pos):
    msd_bm = np.empty(shape=(0))
    fig_met_bm_msd = go.Figure()
    bm_df = trayectoria_browniana_2d(n_steps, speed, [x_pos, y_pos])

    for tau in range(1, bm_df.shape[0]):
        met_df = ([(np.linalg.norm(bm_df.iloc[i-tau]-bm_df.iloc[i]))**2
                   for i in range(tau, bm_df.shape[0],1)])
        d = np.mean(met_df)
        msd_bm = np.append(msd_bm, d)
        msd_bm_df = pd.DataFrame()
        msd_bm_df['msd_bm'] = msd_bm

    fig_met_bm_msd.add_trace(go.Scatter(
        x = np.arange(len(msd_bm_df.index)),
        y = msd_bm_df.msd_bm,
        marker = dict(size=2),
        line = dict(color='#880E4F', width=2),
        mode = 'lines',
        name = 'MSD',
        showlegend = True))

    return fig_met_bm_msd


# MSD Cauchy coefficient
@pn.depends(n_steps, speed, x_pos, y_pos, crw_exp)
def plot_crw_msd(n_steps, speed, x_pos, y_pos, crw_exp):
    msd_crw = np.empty(shape=(0))
    fig_met_crw_msd = go.Figure()
    crw_df = crwv1(n_steps=n_steps, speed=speed, s_pos=[x_pos, y_pos], crw_exp=crw_exp)

    for tau in range(1, crw_df.shape[0]):
        met_df = ([(np.linalg.norm(crw_df.iloc[i-tau]-crw_df.iloc[i]))**2
                   for i in range(tau, crw_df.shape[0],1)])
        d = np.mean(met_df)
        msd_crw = np.append(msd_crw, d)
        msd_crw_df = pd.DataFrame()
        msd_crw_df['msd_crw'] = msd_crw

    fig_met_crw_msd.add_trace(go.Scatter(
        x = np.arange(len(msd_crw_df.index)),
        y = msd_crw_df.msd_crw,
        marker = dict(size=2),
        line = dict(color='#880E4F', width=2),
        mode = 'lines',
        name = 'MSD',
        showlegend = True))

    return fig_met_crw_msd

# MSD LVF
@pn.depends(n_steps, speed, x_pos, y_pos, crw_exp, lv_exp)
def plot_lvf_msd(n_steps, speed, x_pos, y_pos, crw_exp, lv_exp):
    msd_lv = np.empty(shape=(0))
    fig_met_lvf_msd = go.Figure()
    lvf_df = lvfv1(n_steps, speed, [x_pos, y_pos], crw_exp, lv_exp)

    for tau in range(1, lvf_df.shape[0]):
        met_df = ([(np.linalg.norm(lvf_df.iloc[i-tau]-lvf_df.iloc[i]))**2
                   for i in range(tau, lvf_df.shape[0],1)])
        d = np.mean(met_df)
        msd_lv = np.append(msd_lv, d)
        msd_lv_df = pd.DataFrame()
        msd_lv_df['msd_lvf'] = msd_lv

    fig_met_lvf_msd.add_trace(go.Scatter(
        x = np.arange(len(msd_lv_df.index)),
        y = msd_lv_df.msd_lvf,
        marker = dict(size=2),
        line = dict(color='#880E4F', width=2),
        mode = 'lines',
        name = 'Msd',
        showlegend = True))

    return fig_met_lvf_msd


In [7]:
pn.Column(plot_lvf_msd)

Column(sizing_mode='stretch_width')
    [0] ParamFunction(function, _pane=Plotly, defer_load=False, sizing_mode='stretch_width')

# Creacion del Dashboard

In [8]:
# Creacion de los paneles para la observacion de metricas y trayectorias
trayectoria_pane = pn.pane.Plotly()
metrica_pane = pn.pane.Markdown()
trayectoria_3d_pane = pn.pane.Plotly()

# Creacion de seleccionadores para la eleccion del tipo de trayectoria y metricas
metrica_selector = pnw.RadioButtonGroup(options=['Path Length', 'Mean Squared Displacement'], value='Path Length', name='Métrica')
trayectoria_selector = pnw.RadioButtonGroup(options=['Movimiento Browniano', 'Caminata Aleatoria Correlacionada', 'Vuelo de Levy'], value='Movimiento Browniano', name='Tipo de Trayectoria')
trayectoria_selector = pnw.RadioButtonGroup(name='Tipo de Trayectoria', options=['Movimiento Browniano', 'Caminata Aleatoria Correlacionada', 'Vuelo de Levy'])

# Creacion de funciones para realizar actualizaciones de las observaciones y metricas
def actualizar_trayectoria(event):
    tipo_trayectoria = trayectoria_selector.value
    if tipo_trayectoria == 'Movimiento Browniano':
        trayectoria_df = trayectoria_browniana_2d(n_steps.value, speed.value, [x_pos.value, y_pos.value])
    elif tipo_trayectoria == 'Caminata Aleatoria Correlacionada':
        trayectoria_df = crwv1(n_steps.value, speed.value, [x_pos.value, y_pos.value], crw_exp.value)
    elif tipo_trayectoria == 'Vuelo de Levy':
        trayectoria_df = lvfv1(n_steps.value, speed.value, [x_pos.value, y_pos.value], crw_exp.value, lv_exp.value)

    fig = go.Figure(data=go.Scatter(x=trayectoria_df['x_pos'], y=trayectoria_df['y_pos'], mode='lines'))
    fig.update_layout(title=f'Trayectoria - {tipo_trayectoria}', xaxis_title='X', yaxis_title='Y')
    trayectoria_pane.object = fig

def actualizar_metrica(event):
    tipo_metrica = metrica_selector.value
    tipo_trayectoria = trayectoria_selector.value
    if tipo_trayectoria == 'Movimiento Browniano':
        trayectoria_df = trayectoria_browniana_2d(n_steps.value, speed.value, [x_pos.value, y_pos.value])
    elif tipo_trayectoria == 'Caminata Aleatoria Correlacionada':
        trayectoria_df = crwv1(n_steps.value, speed.value, [x_pos.value, y_pos.value], crw_exp.value)
    elif tipo_trayectoria == 'Vuelo de Levy':
        trayectoria_df = lvfv1(n_steps.value, speed.value, [x_pos.value, y_pos.value], crw_exp.value, lv_exp.value)

    if tipo_metrica == 'Path Length':
        distancia = np.linalg.norm(trayectoria_df[['x_pos', 'y_pos']].diff().dropna(), axis=1)
        valor_metrica = np.sum(distancia)
    elif tipo_metrica == 'Mean Squared Displacement':
        desplazamiento = np.linalg.norm(trayectoria_df[['x_pos', 'y_pos']].diff().dropna(), axis=1)**2
        valor_metrica = np.mean(desplazamiento)

    metrica_pane.object = f'Valor de la métrica {tipo_metrica}: {valor_metrica}'

def actualizar_trayectoria_3d(event):
    tipo_trayectoria = trayectoria_selector.value
    if tipo_trayectoria == 'Movimiento Browniano':
        trayectoria_df = trayectoria_browniana_2d(n_steps.value, speed.value, [x_pos.value, y_pos.value])
    elif tipo_trayectoria == 'Caminata Aleatoria Correlacionada':
        trayectoria_df = crwv1(n_steps.value, speed.value, [x_pos.value, y_pos.value], crw_exp.value)
    elif tipo_trayectoria == 'Vuelo de Levy':
        trayectoria_df = lvfv1(n_steps.value, speed.value, [x_pos.value, y_pos.value], crw_exp.value, lv_exp.value)

    fig = go.Figure(data=[go.Scatter3d(x=trayectoria_df['x_pos'], y=trayectoria_df['y_pos'], z=trayectoria_df.index, mode='lines')])
    fig.update_layout(title=f'Trayectoria 3D - {tipo_trayectoria}', scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Paso'))
    trayectoria_3d_pane.object = fig

# Creacion de funciones para manipular cambios en la trayectoria seleccionada
def actualizar_tipo_trayectoria(event):
    tipo_trayectoria = event.new
    if tipo_trayectoria == 'Movimiento Browniano':
        # Ocultar widgets específicos de otras trayectorias y mostrar los de Movimiento Browniano
        crw_exp_panel.visible = False
        lv_exp_panel.visible = False
    elif tipo_trayectoria == 'Caminata Aleatoria Correlacionada':
        # Mostrar el widget para el coeficiente Cauchy y ocultar el de Vuelo de Levy
        crw_exp_panel.visible = True
        lv_exp_panel.visible = False
    elif tipo_trayectoria == 'Vuelo de Levy':
        # Mostrar el widget para el coeficiente Cauchy y el de Vuelo de Levy
        crw_exp_panel.visible = True
        lv_exp_panel.visible = True


# Creacion del widget para el coeficiente Cauchy
crw_exp_panel = pn.Column(pnw.FloatInput(name='Cauchy coefficient', value=0.6, step=0.1, start=0.1, end=1))
# Definir el widget para el exponente del Vuelo de Levy
lv_exp_panel = pn.Column(pnw.FloatInput(name='Levy exponent', value=0.4, step=0.1, start=0.1, end=1))

# Creacion de función para manipular los cambios en la trayectoria seleccionada
def actualizar_tipo_trayectoria(event):
    tipo_trayectoria = event.new
    if tipo_trayectoria == 'Movimiento Browniano':
        # Ocultar widgets específicos de otras trayectorias y mostrar los de Movimiento Browniano
        crw_exp_panel.visible = False
        lv_exp_panel.visible = False
    elif tipo_trayectoria == 'Caminata Aleatoria Correlacionada':
        # Ocultar widgets específicos de otras trayectorias y mostrar los de Caminata Aleatoria Correlacionada
        crw_exp_panel.visible = True
        lv_exp_panel.visible = False
    elif tipo_trayectoria == 'Vuelo de Levy':
        # Ocultar widgets específicos de otras trayectorias y mostrar los de Levy flight
        crw_exp_panel.visible = True
        lv_exp_panel.visible = True

# Asignacion de Callbacks
n_steps.param.watch(actualizar_trayectoria, 'value')
speed.param.watch(actualizar_trayectoria, 'value')
x_pos.param.watch(actualizar_trayectoria, 'value')
y_pos.param.watch(actualizar_trayectoria, 'value')
crw_exp.param.watch(actualizar_trayectoria, 'value')
lv_exp.param.watch(actualizar_trayectoria, 'value')

metrica_selector.param.watch(actualizar_metrica, 'value')
trayectoria_selector.param.watch(actualizar_trayectoria, 'value')
trayectoria_selector.param.watch(actualizar_trayectoria_3d, 'value')
trayectoria_selector.param.watch(actualizar_tipo_trayectoria, 'value')
trayectoria_selector.param.watch(actualizar_tipo_trayectoria, 'value')


# Definicion del dashboard
dashboard = pn.Column(
    pn.Row(
        pn.Column(
            trayectoria_selector,
            n_steps, speed, x_pos, y_pos, crw_exp, lv_exp
        ),
        pn.Column(
            trayectoria_pane,
            trayectoria_3d_pane,
            metrica_pane,
            metrica_selector
        )
    )
)

# Mostrar el dashboard
dashboard.servable()



Column(sizing_mode='stretch_width')
    [0] Row(sizing_mode='stretch_width')
        [0] Column(sizing_mode='stretch_width')
            [0] RadioButtonGroup(name='Tipo de Trayectoria', options=['Movimiento Browniano', ...], sizing_mode='stretch_width', value='Movimiento Browniano')
            [1] IntInput(end=1000, name='Number steps', sizing_mode='stretch_width', start=0, value=50)
            [2] IntInput(end=10, name='Speed', sizing_mode='stretch_width', start=0, value=6)
            [3] FloatInput(end=10000, name='Starting pos_x', sizing_mode='stretch_width', start=0, step=1)
            [4] FloatInput(end=10000, name='Starting pos_y', sizing_mode='stretch_width', start=0, step=1)
            [5] FloatInput(end=1, name='Cauchy Coefficient', sizing_mode='stretch_width', start=0.1, value=0.6)
            [6] FloatInput(end=1, name='Levy exponent', sizing_mode='stretch_width', start=0.1, value=0.4)
        [1] Column(sizing_mode='stretch_width')
            [0] Plotly(None, sizing_mode='stretch_width')
            [1] Plotly(None, sizing_mode='stretch_width')
            [2] Markdown(None, sizing_mode='stretch_width')
            [3] RadioButtonGroup(name='Métrica', options=['Path Length', ...], sizing_mode='stretch_width', value='Path Length')